<a href="https://colab.research.google.com/github/archana3001/DeepLearning_Beginner/blob/master/minor_project_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Implement ANN**

**HyperParameters Optimisation**

Here we will be choosing optimal values for these hyperparameters :

 these hyperparameters are :
1. learning rate
2. no of hidden layers ( no of layers we want between input layer and output layer )
2. no of neurons ( no of nodes we want in each layers)
3. weight ( using weight initialization )
4. batch ( finding batch size for mini batch sdg )
5. momentum ( finding momentum )


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  after removing the cwd from sys.path.


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/AILAB/Constraint_English_Train - Constraint_English_Train.csv')

In [ ]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


**Step 1 : Here we performing vectorisation on our dataset**

Method 1 : WordEmbedding 

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import one_hot
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#text preprocessing and cleaning
corpus=[]
for i in range(0, len(df)):
  tweet=re.sub('[^a-zA-Z]', ' ',df['tweet'][i])
  tweet=tweet.lower()
  tweet=tweet.split()
  tweet=[word for word in tweet if not word in stopwords.words('english')]
  tweet=' '.join(tweet)
  corpus.append(tweet)

#one hot representation
voc_size=10000
one_hot_rep=[one_hot(words, voc_size) for words in corpus]

#embedding
embedded_doc=pad_sequences(one_hot_rep, padding='pre', maxlen=50) #we are making each sentences of word 50 if len(sentence)<50 we add 0 as prefix words
dim=100  #here dimension is no of features we want for embedding
model=Sequential()
model.add(Embedding(voc_size, dim, input_length=50)) #vocab_size, dimension and sentence length are parameters
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # compile the model
#print(model.summary())  # summarize the model

# let's see how our embedded doc is fitted to embedding model 
X=embedded_doc
label_encoder = preprocessing.LabelEncoder()
df['label']= label_encoder.fit_transform(df['label'])
y=df['label']
#print(y.shape)
#print(X.shape)



In [ ]:
model.fit(X, y, epochs=50, verbose=0) # fit the model
loss, accuracy = model.evaluate(X, y, verbose=0) # evaluate the model
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


**train test split**

In [ ]:
#Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.25, random_state=0)

**Deciding no of hidden layers and neurons in each layer**

In [ ]:
def build_model(hyperparams):
  modelhp=keras.Sequential()
  #we are selecting no of hidden layers layers between 2 to 30
  for i in range(hyperparams.Int('num_layers', 2, 30)):
    #inside each layer we are defining no of neurons that is here 4 to 512
    #we are selecting relu as activation function for hidden layers
    modelhp.add(layers.Dense(units=hyperparams.Int('units_'+str(i), min_value=4, max_value=512, step=32),activation='relu'))

  modelhp.add(layers.Dense(1, activation='linear')) # adding one output layer and activation function as sigmoid as it is a classification problem
  modelhp.compile(optimizer=keras.optimizers.Adam(hyperparams.Choice('learning rate',[1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7])),loss='binary_crossentropy', metrics=['accuracy'])
  return modelhp

In [ ]:
from kerastuner.tuners import RandomSearch
#We use objective to specify the objective to select the best models, and we use max_trials to specify the number of different models to try.
tuner=RandomSearch(build_model, objective='val_accuracy', max_trials=20)
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
best_model = tuner.get_best_models()[0]

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 28
units_0: 4
units_1: 228
learning rate: 0.0001
units_2: 484
units_3: 420
units_4: 164
units_5: 164
units_6: 228
units_7: 132
units_8: 4
units_9: 36
units_10: 164
units_11: 452
units_12: 356
units_13: 132
units_14: 36
units_15: 420
units_16: 228
units_17: 484
units_18: 484
units_19: 260
units_20: 36
units_21: 132
units_22: 484
units_23: 388
units_24: 228
units_25: 484
units_26: 452
units_27: 4
Score: 0.6922118663787842
Trial summary
Hyperparameters:
num_layers: 23
units_0: 484
units_1: 324
learning rate: 1e-06
units_2: 452
units_3: 4
units_4: 260
units_5: 260
units_6: 228
units_7: 68
units_8: 228
units_9: 452
units_10: 388
units_11: 164
units_12: 196
units_13: 228
units_14: 36
units_15: 196
units_16: 196
units_17: 452
units_18: 164
units_19: 356
units_20: 68
units_21: 228
units_22: 356
units_23: 324
units_24: 68
units_25: 36
un

**Applying ANN to embedded data after selecting suitable hyperparameters**

In [ ]:
#Scaling test and train data
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [ ]:
X_train.shape

(4815, 50)

**Single hidden layer**

In [ ]:

classifier = Sequential() #create sequential model named as classifier
#units defines the size of the output from the dense layer
classifier.add(Dense(units=1, kernel_initializer='he_normal', activation='relu', input_dim=50)) #input layer
classifier.add(Dense(units=4, kernel_initializer='he_normal', activation='relu')) # 1 hidden layer with 
classifier.add(Dense(units=1, kernel_initializer='he_normal', activation='sigmoid')) # 1 output layer
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) #compile simple ANN
model_history = classifier.fit(X_train, y_train, validation_split=0.33, batch_size=10, epochs=21)


Epoch 1/21
323/323 [==============================] - 1s 3ms/step - loss: 0.6839 - accuracy: 0.6438 - val_loss: 0.6611 - val_accuracy: 0.6394
Epoch 2/21
323/323 [==============================] - 1s 3ms/step - loss: 0.6422 - accuracy: 0.6723 - val_loss: 0.6354 - val_accuracy: 0.6715
Epoch 3/21
323/323 [==============================] - 1s 2ms/step - loss: 0.6173 - accuracy: 0.6990 - val_loss: 0.6178 - val_accuracy: 0.6847
Epoch 4/21
323/323 [==============================] - 1s 3ms/step - loss: 0.5997 - accuracy: 0.7126 - val_loss: 0.6072 - val_accuracy: 0.6941
Epoch 5/21
323/323 [==============================] - 1s 2ms/step - loss: 0.5880 - accuracy: 0.7207 - val_loss: 0.5997 - val_accuracy: 0.7048
Epoch 6/21
323/323 [==============================] - 1s 2ms/step - loss: 0.5806 - accuracy: 0.7263 - val_loss: 0.5960 - val_accuracy: 0.7067
Epoch 7/21
323/323 [==============================] - 1s 3ms/step - loss: 0.5753 - accuracy: 0.7241 - val_loss: 0.5920 - val_accuracy: 0.7067
Epoch 

In [ ]:
#making prediction and evaluating simple ANN
y_pred_single_layer=classifier.predict(X_test)
y_pred_single_layer=(y_pred_single_layer>0.5)

In [ ]:
#y_new=y_pred_single_layer.flatten()

In [ ]:
#creating confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm=confusion_matrix(y_test, y_pred_single_layer)

#calculating accuracy
score=accuracy_score(y_pred_single_layer, y_test)

In [ ]:
print(cm)
print(score)

[[505 267]
 [196 637]]
0.7115264797507788


**building and checking summary of best_model we get after tuning**

In [ ]:
X_train.shape
best_model.build(X_train.shape)
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (4815, 4)                 204       
                                                                 
 dense_1 (Dense)             (4815, 228)               1140      
                                                                 
 dense_2 (Dense)             (4815, 484)               110836    
                                                                 
 dense_3 (Dense)             (4815, 420)               203700    
                                                                 
 dense_4 (Dense)             (4815, 164)               69044     
                                                                 
 dense_5 (Dense)             (4815, 164)               27060     
                                                                 
 dense_6 (Dense)             (4815, 228)               3

**fitting our training data to best_model and predicting for test data and then calculating accuracy**

In [ ]:
best_model.fit(X_train, y_train,epochs=34,batch_size=10)

Epoch 1/34
482/482 [==============================] - 12s 22ms/step - loss: 0.6431 - accuracy: 0.6307
Epoch 2/34
482/482 [==============================] - 10s 21ms/step - loss: 0.6109 - accuracy: 0.6891
Epoch 3/34
482/482 [==============================] - 10s 21ms/step - loss: 0.6000 - accuracy: 0.7067
Epoch 4/34
482/482 [==============================] - 10s 22ms/step - loss: 0.6180 - accuracy: 0.6827
Epoch 5/34
482/482 [==============================] - 10s 22ms/step - loss: 0.5788 - accuracy: 0.7178
Epoch 6/34
482/482 [==============================] - 10s 22ms/step - loss: 0.5735 - accuracy: 0.7227
Epoch 7/34
482/482 [==============================] - 10s 22ms/step - loss: 0.5659 - accuracy: 0.7221
Epoch 8/34
482/482 [==============================] - 10s 21ms/step - loss: 0.5648 - accuracy: 0.7205
Epoch 9/34
482/482 [==============================] - 10s 21ms/step - loss: 0.5560 - accuracy: 0.7302
Epoch 10/34
482/482 [==============================] - 10s 22ms/step - loss: 0.559

In [ ]:
y_best_pred=best_model.predict(X_test)
y_best_pred=y_best_pred.flatten()
y_best_pred=(y_best_pred>0.5)

In [ ]:
#confusion matrix and accuracy score for best_model we get after hyperparameter tuning in keras
cm_best=confusion_matrix(y_test, y_best_pred)

#calculating accuracy
score_best=accuracy_score(y_best_pred, y_test)

print("confusion matrix : \n", cm_best)
print("score : ",score_best)

confusion matrix : 
 [[473 299]
 [146 687]]
score :  0.7227414330218068
